In [26]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [27]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon #multiplies with std

In [41]:
input_dim = 10
latent_dim = input_dim

encoder_inputs = keras.Input(shape=(input_dim,))
#x = layers.Dense(512, activation="relu")(encoder_inputs)
#x = keras.layers.Dropout(0.25)(x)
x = layers.Dense(256, activation="relu")(encoder_inputs)
#x = keras.layers.Dropout(0.25)(x)
#x = layers.Dense(128, activation="relu")(x)
#x = keras.layers.Dropout(0.25)(x)
x = layers.Dense(64, activation="relu")(x)
#x = keras.layers.Dropout(0.25)(x)
x = layers.Dense(32, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

latent_inputs = keras.Input(shape=(latent_dim,))
#x = layers.Dense(32, activation="relu")(latent_inputs)
#x = keras.layers.Dropout(0.25)(x)
x = layers.Dense(64, activation="relu")(x)
#x = keras.layers.Dropout(0.25)(x)
#x = layers.Dense(128, activation="relu")(x)
#x = keras.layers.Dropout(0.25)(x)
x = layers.Dense(256, activation="relu")(x)
#x = keras.layers.Dropout(0.25)(x)
#x = layers.Dense(512, activation="relu")(x)
decoder_outputs = layers.Dense(input_dim, activation="sigmoid")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
dense_23 (Dense)                (None, 256)          2816        input_7[0][0]                    
__________________________________________________________________________________________________
dense_24 (Dense)                (None, 64)           16448       dense_23[0][0]                   
__________________________________________________________________________________________________
dense_25 (Dense)                (None, 32)           2080        dense_24[0][0]                   
____________________________________________________________________________________________

ValueError: Graph disconnected: cannot obtain value for tensor Tensor("input_7:0", shape=(None, 10), dtype=float32) at layer "dense_23". The following previous layers were accessed without issue: []

In [42]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        #mse = tf.keras.losses.MeanSquaredError()
        kl = tf.keras.losses.KLDivergence()
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = kl(data, reconstruction)
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + 0*kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [43]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

dataset_dir = "./Datasets/houseprices_ready.csv"
df = pd.read_csv(dataset_dir, index_col=0)
df = df.fillna(df.mean())

df = df.sample(frac=1, random_state=41)
col_num = len(df.columns)
row_num = len(df.index)

trainx = df.iloc[:int(0.8*row_num), 1:(col_num)]
trainy = df.iloc[:int(0.8*row_num), 0]

testx = df.iloc[int(0.8*row_num):, 1:(col_num)]
testy = df.iloc[int(0.8*row_num):, 0]

scaler = StandardScaler()
trainx_scaled = pd.DataFrame(scaler.fit_transform(trainx), columns = trainx.columns, index = trainx.index)
testx_scaled = pd.DataFrame(scaler.transform(testx), columns = testx.columns, index = testx.index)

In [44]:
trainx_scaled

,LotArea,OverallQual,OverallCond,TotalBsmtSF,FullBath,HalfBath,BedroomAbvGr,TotRmsAbvGrd,Fireplaces,GarageArea
335,14.391355,-0.810781,0.354888,1.012200,0.770659,-0.756619,0.157079,0.276255,2.126222,0.249948
1074,-0.184336,0.646130,-0.536529,0.427102,0.770659,-0.756619,-1.066042,-0.943654,-0.970321,1.638311
933,-0.190779,0.646130,-0.536529,1.014459,0.770659,-0.756619,0.157079,-0.333699,-0.970321,0.441608
982,-0.686115,0.646130,-0.536529,0.485838,0.770659,-0.756619,-1.066042,-0.333699,0.577951,-0.409173
921,-0.163701,-0.810781,1.246305,0.499392,0.770659,3.188607,1.380199,1.496164,-0.970321,-2.222927
...,...,...,...,...,...,...,...,...,...,...
1018,0.023696,0.646130,-0.536529,-1.506656,0.770659,1.215994,0.157079,0.276255,0.577951,-0.343729
1016,0.126311,0.646130,-0.536529,1.023495,0.770659,-0.756619,0.157079,-0.333699,0.577951,0.011542
307,-0.243721,-0.082326,1.246305,-1.653495,-1.047785,-0.756619,0.157079,-0.333699,-0.970321,-2.222927
1450,-0.142879,-0.810781,-0.536529,-0.350016,0.770659,3.188607,1.380199,0.886209,-0.970321,-2.222927


In [45]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(trainx_scaled, epochs=300, batch_size=32)

Epoch 1/300
37/37 [==============================] - 0s 3ms/step - loss: 46.3627 - reconstruction_loss: 46.1634 - kl_loss: 0.2035
Epoch 2/300
37/37 [==============================] - 0s 3ms/step - loss: 46.8845 - reconstruction_loss: 46.1059 - kl_loss: 0.2035
Epoch 3/300
37/37 [==============================] - 0s 3ms/step - loss: 45.0545 - reconstruction_loss: 45.9772 - kl_loss: 0.2018
Epoch 4/300
37/37 [==============================] - 0s 3ms/step - loss: 46.7814 - reconstruction_loss: 46.1024 - kl_loss: 0.2026
Epoch 5/300
37/37 [==============================] - 0s 3ms/step - loss: 46.9866 - reconstruction_loss: 46.0781 - kl_loss: 0.2028
Epoch 6/300
37/37 [==============================] - 0s 3ms/step - loss: 45.7361 - reconstruction_loss: 46.0131 - kl_loss: 0.2033
Epoch 7/300
37/37 [==============================] - 0s 3ms/step - loss: 45.2082 - reconstruction_loss: 46.0375 - kl_loss: 0.2027
Epoch 8/300
37/37 [==============================] - 0s 3ms/step - loss: 46.2499 - reconst

In [ ]:
vae.encoder.compile()
latents = vae.encoder.predict(trainx_scaled)
z_means = latents[0]
pd.DataFrame(z_means)

In [181]:
reconstructed_data = vae.decoder.predict(z_means)
df_reconstructed_data = pd.DataFrame(reconstructed_data)
df_reconstructed_data

,0,1,2,3,4,5,6,7,8,9
0,0.076902,0.095586,0.040753,0.105705,0.096378,0.045957,0.071163,0.130478,0.065411,0.067487
1,0.076902,0.095586,0.040753,0.105705,0.096378,0.045957,0.071163,0.130478,0.065411,0.067487
2,0.076902,0.095586,0.040753,0.105705,0.096378,0.045957,0.071163,0.130478,0.065411,0.067487
3,0.076902,0.095586,0.040753,0.105705,0.096378,0.045957,0.071163,0.130478,0.065411,0.067487
4,0.076902,0.095586,0.040753,0.105705,0.096378,0.045957,0.071163,0.130478,0.065411,0.067487
...,...,...,...,...,...,...,...,...,...,...
1163,0.076902,0.095586,0.040753,0.105705,0.096378,0.045957,0.071163,0.130478,0.065411,0.067487
1164,0.076902,0.095586,0.040753,0.105705,0.096378,0.045957,0.071163,0.130478,0.065411,0.067487
1165,0.076902,0.095586,0.040753,0.105705,0.096378,0.045957,0.071163,0.130478,0.065411,0.067487
1166,0.076902,0.095586,0.040753,0.105705,0.096378,0.045957,0.071163,0.130478,0.065411,0.067487


In [182]:
import matplotlib.pyplot as plt


def plot_latent_space(vae, n=30, figsize=15):
    # display a n*n 2D manifold of digits
    digit_size = 28
    scale = 1.0
    figure = np.zeros((digit_size * n, digit_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-scale, scale, n)
    grid_y = np.linspace(-scale, scale, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = vae.decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[
                i * digit_size : (i + 1) * digit_size,
                j * digit_size : (j + 1) * digit_size,
            ] = digit

    plt.figure(figsize=(figsize, figsize))
    start_range = digit_size // 2
    end_range = n * digit_size + start_range
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.imshow(figure, cmap="Greys_r")
    plt.show()


plot_latent_space(vae)

ValueError: in user code:

    /home/yefe/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1462 predict_function  *
        return step_function(self, iterator)
    /home/yefe/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1452 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/yefe/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/yefe/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/yefe/.local/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/yefe/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/training

In [150]:
ccet = tf.keras.losses.CategoricalCrossentropy()
ccet(np.asarray([1,1,1,1]), np.asarray([3,3,3,3]))

InvalidArgumentError: cannot compute Mul as input #1(zero-based) was expected to be a int64 tensor but is a double tensor [Op:Mul]